# Retrieval-Augmented Generation with Cohere language models on Amazon Bedrock and Weaviate vector database on AWS Market place

The example use case generates targeted advertisements for vacation stay listings based on a target audience. The goal is to use the user query for the target audience (e.g., “family with small children”) to retrieve the most relevant vacation stay listing (e.g., a listing with playgrounds close by) and then to generate an advertisement for the retrieved listing tailored to the target audience.

Note that the following code uses the `v3` Weaviate Python client. A `v4` Weaviate Python client, which uses gRPC under the hood, is being developed (currently in beta as of November 2023). You can find the same RAG pipeline using the new `v4` client in the related GitHub repository.

This notebook should work well with the Data Science 3.0 kernel in SageMaker Studio.

## Dataset Overview
The dataset is available from [Inside AirBnB](http://data.insideairbnb.com/the-netherlands/north-holland/amsterdam/2023-09-03/data/listings.csv.gz) and is licensed under a [Creative Commons Attribution 4.0 International License](https://creativecommons.org/licenses/by/4.0/).

Download the data and save it in a folder called `data`.

In [ ]:
!wget http://data.insideairbnb.com/the-netherlands/north-holland/amsterdam/2023-09-03/data/listings.csv.gz

In [ ]:
!gzip -d listings.csv.gz
!mkdir data
!mv listings.csv ./data

In [1]:
import pandas as pd
import json

# Read CSV file
csv_file = './data/listings.csv'
df = pd.read_csv(csv_file, usecols=['host_name',
                                    'property_type',
                                    'description',
                                    'neighborhood_overview',
                                    ])

df.fillna('Unknown', inplace=True)

display(df.head())

,description,neighborhood_overview,host_name,property_type
0,"Really quiet, spacious and safe, a nice place ...","The neighborhood is very green, quiet, safe an...",Xsjong,Private room in condo
1,Our cool and spacious loft is perfect for a st...,Our neighbourhood is ons of the most beautiful...,J & R,Entire rental unit
2,Room to rent in my houseboat. The room has a p...,It is just outside the Jordan in between the c...,Nicole,Private room in boat
3,"Spacious houseboat in Amsterdam, suitable for ...",The houseboat lies in an area with ± 200 house...,Danielle,Houseboat
4,Royal Bed & Coffee Room with a very comfortabl...,The building is located in Amsterdam centre in...,Marcel,Private room in rental unit


## Prerequisites
To be able to follow along and use any AWS services in the following tutorial, please make sure you have an [AWS account](https://signin.aws.amazon.com/signin?redirect_uri=https%3A%2F%2Fportal.aws.amazon.com%2Fbilling%2Fsignup%2Fresume&client_id=signup).

## Step 1: Enable components of the AI-native technology stack
First, you will need to enable the relevant components discussed in the solution overview in your AWS account. 
First, enable access to the Cohere Command and Embed foundation models available on Amazon Bedrock through the AWS Management Console. Navigate to the Model access page, click on Edit, and select the foundation models of your choice.

Next, set up a Weaviate cluster. First, subscribe to the [Weaviate Kubernetes Cluster on AWS Marketplace](https://aws.amazon.com/marketplace/pp/prodview-cicacyv63r43i). Then, launch the software using a [CloudFormation template according to your preferred availability zone](https://weaviate.io/developers/weaviate/installation/aws-marketplace#aws-marketplace). The CloudFormation template is pre-filled with default values. To follow along in this guide, edit the following fields: 
* Stack name: Enter a stack name
* Authentication: It is recommended to enable authentication by setting helmauthenticationtype to apikey and defining a helmauthenticationapikey.
* Enabled modules: Make sure “tex2vec-aws” and “generative-aws” are present in the list of enabled modules within Weaviate. 

This template takes about 30 minutes to complete.

## Step 2: Connect to Weaviate
On the SageMaker console, navigate to Notebook instances and create a new notebook instance. 

Then, install the Weaviate client package with the required dependencies:

In [2]:
# pip install weaviate-client

Now, you can connect to your Weaviate instance with the following code. You can find the relevant information as follows:
* Weaviate URL: Access Weaviate via the load balancer URL. Go to the Services section of AWS, under EC2 > Load Balancers find the load balancer, and look for the DNS name column.
* Weaviate API Key: This is the key you set earlier in the CloudFormation template (helmauthenticationapikey). 
* AWS Access Key: You can retrieve the access keys for your user in the AWS Identity and Access Management (IAM) Console.

In [3]:
import weaviate

client = weaviate.Client(
    url="<YOUR-WEAVIATE-URL>",
    auth_client_secret=weaviate.AuthApiKey(
        api_key="<YOUR-WEAVIATE-API-KEY>"),
    additional_headers={
        "X-AWS-Access-Key": "<AWS-ACCESS-KEY>",
        "X-AWS-Access-Secret": "<AWS-ACCESS-SECRET>"
    }
)

client.get_meta()

{'hostname': 'http://[::]:8080',
 'modules': {'generative-cohere': {'documentationHref': 'https://docs.cohere.com/reference/generate',
   'name': 'Generative Search - Cohere'},
  'generative-openai': {'documentationHref': 'https://platform.openai.com/docs/api-reference/completions',
   'name': 'Generative Search - OpenAI'},
  'generative-palm': {'documentationHref': 'https://cloud.google.com/vertex-ai/docs/generative-ai/chat/test-chat-prompts',
   'name': 'Generative Search - Google PaLM'},
  'qna-openai': {'documentationHref': 'https://platform.openai.com/docs/api-reference/completions',
   'name': 'OpenAI Question & Answering Module'},
  'ref2vec-centroid': {},
  'reranker-cohere': {'documentationHref': 'https://txt.cohere.com/rerank/',
   'name': 'Reranker - Cohere'},
  'text2vec-cohere': {'documentationHref': 'https://docs.cohere.ai/embedding-wiki/',
   'name': 'Cohere Module'},
  'text2vec-huggingface': {'documentationHref': 'https://huggingface.co/docs/api-inference/detailed_para

## Step 3: Configure the Amazon Bedrock module to enable Cohere models

Next, you will define a data collection (i.e., `class`) called `Listings` to store the listings’ data objects, which is analogous to creating a table in a relational database. In this step, you will configure the relevant modules to enable the usage of Cohere language models hosted on Amazon Bedrock natively from within the Weaviate vector database. The vectorizer (`"text2vec-aws"`) and generative module (` "generative-aws"`) are specified in the data collection definition. Both of these modules take three parameters:
* `"service"`: `"bedrock"` for Amazon Bedrock (Alternatively, `"sagemaker"` for Amazon Sagemaker Jumpstart)
* `"Region"`: The region where your model is deployed
* `"model"`: The foundation model’s name

In [4]:
collection_definition = {
    "class": "Listings",
    "description": "AirBnb Listings",
    "moduleConfig": {
        "text2vec-aws": {
            "service": "bedrock",
            "region": "us-east-1",
            "model": "cohere.embed-english-v3",
            "vectorizeClassName": False,

        },
        "generative-aws": {
            "service": "bedrock",
            "region": "us-east-1",
            "model": "cohere.command-text-v14"
        }
    },
    "vectorizer": "text2vec-aws",
}

## Step 4: Ingest data into the Weaviate vector database 
In this step, you will also define the structure of the data collection by configuring its properties. Aside from the property’s name and data type, you can also configure if only the data object shall be stored or if it shall be stored together with its vector embeddings. In this example, `host_name` and `property_type` are not vectorized.

In [5]:
collection_definition["properties"] = [
    {
        "name": "description",
        "dataType": ["text"],
        "description": "The description of the apartment listing",
    },
    {
        "name": "host_name",
        "dataType": ["text"],
                "description": "The name of the host of the apartment listing",
                "moduleConfig": {
                    "text2vec-aws": {
                        "skip": True,
                        "vectorizePropertyName": False,

                    }
        }
    },
    {
        "name": "neighborhood_overview",
        "dataType": ["text"],
        "description": "The description of  the neighbourhood of the apartment listing",
    },
    {
        "name": "property_type",
                "dataType": ["text"],
                "description": "The type of property of the listing",
                "moduleConfig": {
                    "text2vec-aws": {
                        "skip": True,
                        "vectorizePropertyName": False,
                    }
                }
    },
]

Run the following code to create the collection in your Weaviate instance.

In [6]:
if client.schema.exists("Listings"):
    client.schema.delete_class("Listings")

client.schema.create_class(collection_definition)

client.schema.get("Listings")

{'class': 'Listings',
 'description': 'AirBnb Listings',
 'invertedIndexConfig': {'bm25': {'b': 0.75, 'k1': 1.2},
  'cleanupIntervalSeconds': 60,
  'stopwords': {'additions': None, 'preset': 'en', 'removals': None}},
 'moduleConfig': {'generative-cohere': {},
  'text2vec-cohere': {'vectorizeClassName': False,
   'vectorizePropertyName': False}},
 'multiTenancyConfig': {'enabled': False},
 'properties': [{'dataType': ['text'],
   'description': 'The description of the apartment listing',
   'indexFilterable': True,
   'indexSearchable': True,
   'moduleConfig': {'text2vec-cohere': {'skip': False,
     'vectorizePropertyName': False}},
   'name': 'description',
   'tokenization': 'word'},
  {'dataType': ['text'],
   'description': 'The name of the host of the apartment listing',
   'indexFilterable': True,
   'indexSearchable': True,
   'moduleConfig': {'text2vec-cohere': {'skip': True,
     'vectorizePropertyName': False}},
   'name': 'host_name',
   'tokenization': 'whitespace'},
  {'d

You can now add objects to Weaviate. You will be using a batch import process for maximum efficiency. Run the code below to import data. During the import, Weaviate will use the defined vectorizer to create a vector embedding for each object. The following code loads objects initializes a batch process, and adds objects to the target collection one by one.

In [7]:
from weaviate.util import generate_uuid5

client.batch.configure(batch_size=100)  # Configure batch

# Initialize batch process
with client.batch as batch:
    for _, row in df.iterrows():
        listing_object = {
            "description": row["description"],
            "host_name": row["host_name"],
            "neighborhood_overview": row["neighborhood_overview"],
            "property_type": row["property_type"],
        }
        batch.add_data_object(
            class_name="Listings",
            data_object=listing_object,
            uuid=generate_uuid5(listing_object)
        )

client.query.aggregate("Listings").with_meta_count().do()

{'error': [{'message': "update vector: failed with status: 429 error: You are using a Trial key, which is limited to 100 API calls / minute. You can continue to use the Trial key for free or upgrade to a Production key with higher rate limits at 'https://dashboard.cohere.ai/api-keys'. Contact us on 'https://discord.gg/XW44jPfYJu' or email us at support@cohere.com with any questions"}]}
{'error': [{'message': "update vector: failed with status: 429 error: You are using a Trial key, which is limited to 100 API calls / minute. You can continue to use the Trial key for free or upgrade to a Production key with higher rate limits at 'https://dashboard.cohere.ai/api-keys'. Contact us on 'https://discord.gg/XW44jPfYJu' or email us at support@cohere.com with any questions"}]}
{'error': [{'message': "update vector: failed with status: 429 error: You are using a Trial key, which is limited to 100 API calls / minute. You can continue to use the Trial key for free or upgrade to a Production key wit

{'data': {'Aggregate': {'Listings': [{'meta': {'count': 100}}]}}}

## Step 5: Retrieval-Augmented Generation to generate targeted advertisements

Finally, you can build a RAG pipeline by implementing a generative search query on your Weaviate instance. For this, you will first define a prompt template in the form of an f-string that can take in the user query (`{target_audience}`) directly and the additional context (`{{host_name}}`, `{{property_type}}`, `{{description}}`, `{{neighborhood_overview}}`) from the vector database at runtime.

Next, you will run a generative search query. This prompts the defined generative model with a prompt that is comprised of the user query as well as the retrieved data. The following query retrieves one listing object (`.with_limit(1)`) from the `Listings` collection that is most similar to the user query (`.with_near_text({"concepts": target_audience})`). Then the user query (`target_audience`) and the retrieved listings properties (`["description", "neighborhood", "host_name", "property_type"]`) are fed into the prompt template.

In [8]:
def generate_targeted_ad(target_audience):
    generate_prompt = f"""You are a copywriter.
    Write short advertisement for the following vacation stay.
    Host: {{host_name}}
    Property type: {{property_type}}
    Description: {{description}}
    Neighborhood: {{neighborhood_overview}}
    Target audience: {target_audience}
    """
    result = client.query\
        .get("Listings", ["description", "neighborhood_overview", "host_name", "property_type"])\
        .with_near_text({"concepts": target_audience})\
        .with_limit(1)\
        .with_generate(single_prompt=generate_prompt)\
        .do()

    return result

Below, you can see that the results for the `target_audience = “Family with small children”`.

In [11]:
result = generate_targeted_ad("Families with young children")
# print(json.dumps(result["data"]["Get"]["Listings"], indent=4))
print(result["data"]["Get"]["Listings"][0]
      ["_additional"]["generate"]["singleResult"])

Looking for a kid-friendly home away from home in one of the trendiest areas of Amsterdam? Look no further than this stylish townhouse in the heart of the city! Our 120ft² space is perfect for a family of four or a group of adults, with two bedrooms featuring elevated beds suitable for kids and one bedroom with a single bed. The ground floor features a spacious living room, a kitchen with a large dining table, and a half bath while heading upstairs leads you to a master bedroom and a full bathroom. Our central location means you're just steps away from the best cafes, restaurants, and bars that the city has to offer, and the Vondelpark and other attractions are only a short walk away! Supermarkets and paid parking are also conveniently located nearby. Experience the best of Amsterdam in a laid-back, local way and create unforgettable memories with your family at our cozy townhouse. 

Book now and get ready to explore the dynamic and family-friendly neighborhood of Helmersbuurt!


Here is another example for an elderly couple.

In [12]:
result = generate_targeted_ad("Elderly couple")
# print(json.dumps(result["data"]["Get"]["Listings"], indent=4))
print(result["data"]["Get"]["Listings"][0]
      ["_additional"]["generate"]["singleResult"])

Looking for a quiet and peaceful place to stay during your visit to Amsterdam? Look no further than Gianni's elegant and tidy B&B! Located in a charming family neighborhood, just a stone's throw away from the city center, this private room offers the perfect retreat after a long day of sightseeing. With high-speed WiFi, a private mini fridge, and a shared bathroom, you'll have everything you need for a comfortable and convenient stay. Plus, with a parking spot available for just 14 euros a day (free on Sundays), you can explore the city with ease. And if you're looking to save even more money, the room comes with a microwave and water boiler, so you can easily prepare light meals during your stay. Remember to note that Gianni lives in the same apartment, but don't worry - you'll have plenty of privacy and space to yourselves. 

Book your stay now and enjoy everything Amsterdam has to offer!
